In [1]:
# Importación de librerías
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Carga de los datasets
items_game  = pd.read_csv('../Datacsv/items_games_.csv',  parse_dates=['Release_date'])
review_game = pd.read_csv('../Datacsv/review_games_.csv', parse_dates=['Release_date', 'Posted'])

In [3]:
items_game

,Id,App_name,Release_date,Genres,Playtime_forever,User_id
0,70,Half-Life,1998-11-08,Action,108,doctr
1,2400,The Ship: Murder Party,2006-07-11,"Action, Indie, RPG",1,doctr
2,1520,DEFCON,2006-09-29,"Indie, Strategy",158,evcentric
3,2420,The Ship: Single Player,2006-11-20,"Action, Indie, RPG",15,doctr
4,4000,Garry's Mod,2006-11-29,"Indie, Simulation",412,js41637
...,...,...,...,...,...,...
458375,9940,Blade Kitten,2010-09-22,"Action, Adventure",209,76561198302935315
458376,42700,Call of Duty®: Black Ops,2010-11-09,Action,64,76561198312638244
458377,21100,F.E.A.R. 3,2011-06-21,Action,115,76561198313357718
458378,102200,Runespell: Overture,2011-07-20,"Adventure, Indie, RPG",248,76561198302935315


In [4]:
review_game

,Id,App_name,Release_date,Genres,User_id,Posted,Recommend,Sentiment_analysis
0,70,Half-Life,1998-11-08,Action,EizanAratoFujimaki,2015-10-28,True,2
1,70,Half-Life,1998-11-08,Action,GamerFag,2011-01-27,True,0
2,70,Half-Life,1998-11-08,Action,76561198020928326,2014-07-01,True,2
3,4000,Garry's Mod,2006-11-29,"Indie, Simulation",WeiEDKrSat,2013-12-25,True,1
4,4000,Garry's Mod,2006-11-29,"Indie, Simulation",death-hunter,2014-02-19,True,2
...,...,...,...,...,...,...,...,...
6270,620,Portal 2,2011-04-18,"Action, Adventure",maddydufall,2015-09-26,True,2
6271,620,Portal 2,2011-04-18,"Action, Adventure",76561198209667851,2015-09-04,True,0
6272,113400,APB Reloaded,2011-12-06,"Action, Free to Play, Massively Multiplayer",TripleQMark,2015-07-26,True,2
6273,212500,The Lord of the Rings Online™,2012-06-06,"Free to Play, Massively Multiplayer, RPG",76561198166194627,2014-12-03,True,2


In [5]:
def PlayTimeGenre(genero):
    # 1. Filtrar el DataFrame para incluir solo los juegos del género dado
    juegos_genero = items_game[items_game['Genres'].str.contains(genero, case=False)]

    # 2. Agrupar los datos por año de lanzamiento y calcular la suma de las horas jugadas para cada año
    horas_por_año = juegos_genero.groupby(juegos_genero['Release_date'].dt.year)['Playtime_forever'].sum()

    # 3. Encontrar el año con el máximo total de horas jugadas
    año_mas_horas = horas_por_año.idxmax()

    # 4. Devolver el resultado en el formato requerido
    resultado = {"Año de lanzamiento con más horas jugadas para " + genero: int(año_mas_horas)}

    return resultado

# Ejemplo de uso
print(PlayTimeGenre("Action"))

{'Año de lanzamiento con más horas jugadas para Action': 2011}


In [7]:
def UserForGenre(genero):
    # 1. Filtrar el DataFrame para incluir solo los juegos del género dado
    juegos_genero = items_game[items_game['Genres'].str.contains(genero, case=False)]

    if juegos_genero.empty:
        return "No se encontraron juegos para el género especificado."

    # 2. Agrupar los datos por usuario y año de lanzamiento y calcular la suma de las horas jugadas
    horas_por_usuario_y_año = juegos_genero.groupby(['User_id', juegos_genero['Release_date'].dt.year])['Playtime_forever'].sum()

    # 3. Encontrar el usuario que acumula más horas jugadas para el género dado
    usuario_mas_horas = horas_por_usuario_y_año.groupby(level=0).sum().idxmax()

    # 4. Obtener la acumulación de horas jugadas por año para el usuario encontrado
    horas_por_año_usuario = horas_por_usuario_y_año.loc[usuario_mas_horas].reset_index()
    horas_por_año_usuario.rename(columns={'Release_date': 'Año', 'Playtime_forever': 'Horas'}, inplace=True)
    horas_por_año_usuario = horas_por_año_usuario.to_dict(orient='records')

    # 5. Crear el diccionario de retorno
    resultado = {"Usuario con más horas jugadas para " + genero: usuario_mas_horas, "Horas jugadas": horas_por_año_usuario}

    return resultado

# Ejemplo de uso
print(UserForGenre("Action"))

{'Usuario con más horas jugadas para Action': 'thiefofrosesinlalaland', 'Horas jugadas': [{'Año': 2007, 'Horas': 543}, {'Año': 2008, 'Horas': 1203}, {'Año': 2009, 'Horas': 523}, {'Año': 2010, 'Horas': 39207}, {'Año': 2011, 'Horas': 370751}, {'Año': 2012, 'Horas': 892}, {'Año': 2013, 'Horas': 29}]}


In [8]:
def UsersRecommend(año: int):
    # Filtrar el DataFrame de reseñas para incluir solo las reseñas del año especificado
    reseñas_año = review_game[review_game['Posted'].dt.year == año]

    # Filtrar las reseñas para incluir solo aquellas donde Recommend es verdadero
    reseñas_recomendadas = reseñas_año[reseñas_año['Recommend']]

    # Contar la cantidad de veces que aparece cada juego en las reseñas
    juegos_recomendados = reseñas_recomendadas['App_name'].value_counts()

    # Obtener los nombres de los juegos más recomendados (top 3)
    top_3_juegos = juegos_recomendados.head(3)

    # Crear la lista de juegos más recomendados en el formato deseado
    resultado = [{"Puesto {}".format(index + 1): juego} for index, juego in enumerate(top_3_juegos.index)]

    return resultado

print(UsersRecommend(2014))

[{'Puesto 1': "Garry's Mod"}, {'Puesto 2': "Sid Meier's Civilization® V"}, {'Puesto 3': 'Portal 2'}]


In [10]:
def UsersNotRecommend(año: int):
    # Filtrar el DataFrame de reseñas para incluir solo las reseñas del año especificado
    reseñas_año = review_game[review_game['Posted'].dt.year == año]

    # Filtrar las reseñas para incluir solo aquellas donde Recommend es falso
    reseñas_no_recomendadas = reseñas_año[~reseñas_año['Recommend']]

    # Contar la cantidad de veces que aparece cada juego en las reseñas
    juegos_no_recomendados = reseñas_no_recomendadas['App_name'].value_counts()

    # Obtener los nombres de los juegos menos recomendados (top 3)
    top_3_juegos = juegos_no_recomendados.head(3)

    # Crear la lista de juegos menos recomendados en el formato deseado
    resultado = [{"Puesto {}".format(index + 1): juego} for index, juego in enumerate(top_3_juegos.index)]

    return resultado

print(UsersNotRecommend(2014))

[{'Puesto 1': "Garry's Mod"}, {'Puesto 2': 'Realm of the Mad God'}, {'Puesto 3': 'APB Reloaded'}]


In [11]:
def sentiment_analysis(año: int):
    # Filtrar el DataFrame de reseñas para incluir solo las reseñas del año especificado
    reseñas_año = review_game[review_game['Posted'].dt.year == año]

    # Contar la cantidad de registros para cada categoría de análisis de sentimiento
    count_sentiments = reseñas_año['Sentiment_analysis'].value_counts()

    # Crear el diccionario con el formato deseado
    resultado = {"Positivo": count_sentiments.get(2, 0),
                 "Neutral": count_sentiments.get(1, 0),
                 "Negativo": count_sentiments.get(0, 0)}

    return resultado

print(sentiment_analysis(2014))

{'Positivo': 1785, 'Neutral': 539, 'Negativo': 360}


In [12]:
def recomendacion_juego(id_producto: int):
    # Filtrar el DataFrame para incluir solo los juegos relevantes
    juegos_rel = review_game[review_game['Id'] == id_producto]

    # Eliminar las columnas no deseadas antes de calcular la similitud
    juegos_rel = juegos_rel.drop(columns=['Id', 'App_name', 'User_id', 'Posted', 'Release_date'])

    # Codificar la columna 'Genres' usando codificación ordinal
    encoder_genres = LabelEncoder()
    juegos_rel['Genres'] = encoder_genres.fit_transform(juegos_rel['Genres'])

    # Codificar la columna 'Recommend' usando codificación ordinal
    juegos_rel['Recommend'] = juegos_rel['Recommend'].astype(int)

    # Calcular la matriz de similitud del coseno entre los juegos
    matriz_similitud = cosine_similarity(juegos_rel)

    # Obtener los índices de los juegos más similares al juego dado
    juegos_similares_indices = matriz_similitud.argsort()[0][-6:-1][::-1]

    # Obtener los nombres de los juegos similares
    juegos_similares = review_game.loc[juegos_similares_indices, 'App_name'].tolist()

    return juegos_similares

print(recomendacion_juego(620))

['Fallout 2: A Post Nuclear Role Playing Game', "Garry's Mod", "Garry's Mod", 'BioShock™', 'Command & Conquer: Red Alert 3']


In [13]:
def recomendacion_usuario(User_id: str):
    # Filtrar el DataFrame para incluir solo los juegos relevantes
    juegos_rel = review_game[review_game['User_id'] == User_id]

    # Eliminar las columnas no deseadas antes de calcular la similitud
    juegos_rel = juegos_rel.drop(columns=['Id', 'App_name', 'User_id', 'Posted', 'Release_date', 'Recommend', 'Sentiment_analysis'])

    # Codificar la columna 'Genres' usando codificación ordinal
    encoder_genres = LabelEncoder()
    juegos_rel['Genres'] = encoder_genres.fit_transform(juegos_rel['Genres'])

    # Calcular la matriz de similitud del coseno entre los juegos
    matriz_similitud = cosine_similarity(juegos_rel)

    # Obtener los índices de los juegos más similares al juego dado
    juegos_similares_indices = matriz_similitud.argsort()[0][-6:-1][::-1]

    # Obtener los nombres de los juegos similares
    juegos_similares = review_game.loc[juegos_similares_indices, 'App_name'].tolist()

    return juegos_similares

print(recomendacion_usuario('ShadowYT100'))

['Half-Life']
